In [ ]:
%pip install labelme tensorflow opencv-python matplotlib albumentations 

# 1. Obtenció i tractament de dades

### 1.1 Ús de la llibreria LabelMe

In [ ]:
!labelme 

###  1.2 Creació de la base de dades

In [7]:
#importació de llibreries necessàries
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt
import os
import random
import shutil
import math

In [ ]:
#limitació de la memòria GPU 
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
imatges = tf.data.Dataset.list_files('imatges\\*.jpg') # introduïm les imatges

In [ ]:
def carregar_imatge(imatge): #funció per a carregar imatges
    byte_img = tf.io.read_file(imatge)
    img = tf.image.decode_jpeg(byte_img)
    return img

In [ ]:
imatges = imatges.map(carregar_imatge) #executem la funció 

In [ ]:
imatges.as_numpy_iterator().next() 

In [ ]:
def imatgesDividir(dir_carpeta, n_imatges):
    num_elements = len(os.listdir(os.path.join(dir_carpeta, "imatges")))
    n_train = round(num_elements*0.7) # 70% de les imatges per entrenar
    n_test = round(num_elements*0.15) # 15% de les imatges per provar
    n_val = num_elements - (n_train + n_test) # 15% de les imatges per validar

    llista = []
    for i in range(1, (n_imatges + 1)):
        llista.append(i)

    while len(llista) > n_train:
        if (len(os.listdir(os.path.join(dir_carpeta,"dades","test")))-1)< n_test:
            for i in range(n_test):
                n = random.choice(llista)
                nom = str(str(n) + ".jpg")
                shutil.move(os.path.join(dir_carpeta, "imatges", nom ), os.path.join(dir_carpeta, "dades", "test" , "imatges"))
                llista.remove(n)

        if (len(os.listdir(os.path.join(dir_carpeta,"dades","val")))-1) < n_val:
            for i in range(n_val):
                n = random.choice(llista)
                nom = str(str(n) + ".jpg")
                shutil.move(os.path.join(dir_carpeta, "imatges", nom), os.path.join(dir_carpeta,"dades" , "val", "imatges"))
                llista.remove(n)

    for i in llista:
        nom = str(i) + ".jpg"
        shutil.move(os.path.join(dir_carpeta, "imatges", nom), os.path.join(dir_carpeta, "dades","train", "imatges"))

In [ ]:
# Crear la carpeta principal
os.mkdir("dades")

# Crear les subcarpetes dins de la carpeta principal
for subcarpeta in ["train", "test", "val"]:
    path_subcarpeta = os.path.join("dades", subcarpeta)
    os.mkdir(path_subcarpeta)
    path_subsubcarpeta = os.path.join("dades", subcarpeta, "imatges")
    os.mkdir(path_subsubcarpeta)
    path_subsubcarpeta2 = os.path.join("dades", subcarpeta, "labels")
    os.mkdir(path_subsubcarpeta2)

imatgesDividir('T:\practica_tr',300) #divisió de les carpetes i de les imatges

### Partició de les dades

In [ ]:
# mou les els labels a les seves respectives carpetes
def moureLabels(dir_carpeta):
    for carpeta in ['train','test','val']:
        for arxiu in os.listdir(os.path.join(dir_carpeta,'dades', carpeta, 'imatges')): #per cada arxiu en cada carpeta
            n = arxiu.split(".")
            json = str(n[0] + ".json")
            shutil.move(os.path.join(dir_carpeta, "labels", json ), os.path.join(dir_carpeta, "dades", carpeta, "labels"))#canviem

In [ ]:
moureLabels('T:\practica_tr')

### 1.2.1 Ús de la llibreria Albumentation

In [ ]:
import albumentations as alb
import cv2

In [ ]:
augmentor = alb.Compose([alb.RandomCrop(width=1024, height=1024),
                         alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.5),
                         alb.RandomGamma(p=0.7), 
                         alb.RGBShift(p=0.5), 
                         alb.VerticalFlip(p=0.7)], 
                       bbox_params=alb.BboxParams(format='albumentations', 
                                                  label_fields=['class_labels']))

In [ ]:
os.mkdir("aug_dades")

# Crear les subcarpetes dins de la carpeta principal
for subcarpeta in ["train", "test", "val"]:
    path_subcarpeta = os.path.join("aug_dades", subcarpeta)
    os.mkdir(path_subcarpeta)
    path_subsubcarpeta = os.path.join("aug_dades", subcarpeta, "imatges")
    os.mkdir(path_subsubcarpeta)
    path_subsubcarpeta2 = os.path.join("aug_dades", subcarpeta, "labels")
    os.mkdir(path_subsubcarpeta2)


In [ ]:
classes_fruita = ["buit","poma", "pera", "mandarina"]
partitions = ['train', 'test', 'val']

for partition in partitions:
    input_folder = os.path.join('dades', partition, 'imatges')
    output_folder = os.path.join('aug_dades', partition, 'imatges')

    for imatge in os.listdir(input_folder):
        img_path = os.path.join(input_folder, imatge)
        img = cv2.imread(img_path)
        label_path = os.path.join('dades', partition, 'labels', f'{imatge.split(".")[0]}.json')

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            for shape in label['shapes']:
                class_name = shape['label'] 
                class_id = classes_fruita.index(class_name)
                coords = shape['points']
                coords = [(coords[0][0]/1024), (coords[0][1]/1024), (coords[1][0]/1024), (coords[1][1]/1024)]

                try:
                    for x in range(5):
                        augmented = augmentor(image=img, bboxes=[coords], class_labels=[class_name])
                        augmented_img = augmented['image']

                        annotation = {
                            'image': f'{imatge.split(".")[0]}.{x}.jpg',
                            'bbox': augmented['bboxes'],
                            'class': class_id,
                        }
                        output_img_path = os.path.join(output_folder, annotation['image'])
                        cv2.imwrite(output_img_path, augmented_img)

                        output_json_path = os.path.join('aug_dades', partition, 'labels', f'{imatge.split(".")[0]}.{x}.json')
                        with open(output_json_path, 'w') as f:
                            json.dump(annotation, f) 
                except Exception as e:
                    print(e)


### Incloure imatges creades amb Albumentations a la Dataset

In [ ]:
train_imatges = tf.data.Dataset.list_files('aug_dades\\train\\imatges\\*.jpg',shuffle=False)
train_imatges = train_imatges.map(carregar_imatge)
train_imatges = train_imatges.map(lambda x: tf.image.resize(x, (120,120)))
train_imatges = train_imatges.map(lambda x: x/255)

In [ ]:
test_imatges = tf.data.Dataset.list_files('aug_dades\\test\\imatges\\*.jpg', shuffle=False)
test_imatges = test_imatges.map(carregar_imatge)
test_imatges = test_imatges.map(lambda x: tf.image.resize(x, (120,120)))
test_imatges = test_imatges.map(lambda x: x/255)

In [ ]:
val_imatges = tf.data.Dataset.list_files('aug_dades\\val\\imatges\\*.jpg', shuffle=False)
val_imatges = val_imatges.map(carregar_imatge)
val_imatges = val_imatges.map(lambda x: tf.image.resize(x, (120,120)))
val_imatges = val_imatges.map(lambda x: x/255)

In [ ]:
train_imatges.as_numpy_iterator().next()

### 2.5 Carregar labels

In [ ]:
def carregar_labels(label_path):
    with open(label_path.numpy(), 'r', encoding = "utf-8") as f:
        label = json.load(f)
        
    return [label['class']], label['bbox']

In [ ]:
train_labels = tf.data.Dataset.list_files('aug_dades\\train\\labels\\*.json', shuffle=False)
train_labels = train_labels.map(lambda x: tf.py_function(carregar_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
test_labels = tf.data.Dataset.list_files('aug_dades\\test\\labels\\*.json', shuffle=False)
test_labels = test_labels.map(lambda x: tf.py_function(carregar_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
val_labels = tf.data.Dataset.list_files('aug_dades\\val\\labels\\*.json', shuffle=False)
val_labels = val_labels.map(lambda x: tf.py_function(carregar_labels, [x], [tf.uint8, tf.float16]))

In [ ]:
train_labels.as_numpy_iterator().next()

### Combinar etiquetes i imatges

In [ ]:
num_elements={}
for carpeta in ['train','test','val']:
    x = len(os.listdir(os.path.join('aug_dades', carpeta, 'imatges')))
    num_elements.update({carpeta : round(x * 1.5)})

In [ ]:
train = tf.data.Dataset.zip((train_imatges, train_labels))
train = train.shuffle(num_elements['train'])
train = train.batch(8)
train = train.prefetch(4)

In [ ]:
test = tf.data.Dataset.zip((test_imatges, test_labels))
test = test.shuffle(num_elements['test'])
test = test.batch(8)
test = test.prefetch(4)

In [ ]:
val = tf.data.Dataset.zip((val_imatges, val_labels))
val = val.shuffle(num_elements['val'])
val = val.batch(8)
val = val.prefetch(4)

# 2. Preparació de la intel·ligència artificial

### 2.1 Descarregar i carregar el model VGG16

In [ ]:
import tensorflow.keras.models
import tensorflow.keras.layers
import tensorflow.keras.applications

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D
from tensorflow.keras.applications import VGG16

In [ ]:
vgg = VGG16(include_top=False) 
#Marca que les ultimes capes de la xarxa no les utilitzarem perquè afegirem les nostres pròpies

In [ ]:

'''
La funció crea i retorna un model de xarxa neuronal convolucional per a la detecció dels objectes. 
El model té dues parts: una per a determinar si una imatge conté algun objecte i una altra per a localitzar l'obkecte (poma, pera o mandarina) a l'imatge. 
Al utilitzar VGG16, com que esta pre-entrenada, les característiques del input
i les classifica en dues branques que son les dues parts mencionades abans de manera automàtica. 
'''
def build_model(): 
    input_layer = Input(shape=(250,250,3))
    
    vgg = VGG16(include_top=False)(input_layer)
    #Model de classificació
    f1 = GlobalMaxPooling2D()(vgg)
    class1 = Dense(2048, activation='relu')(f1) #relu == funció que determina la classe; 
    class2 = Dense(1, activation='sigmoid')(class1) # sigmoid == funció que determina la presició de la classe

    # sigmoid = f(x) = 1/(1+e^-x)
    
    #Model de localització de coordenades
    f2 = GlobalMaxPooling2D()(vgg)
    regress1 = Dense(2048, activation='relu')(f2)
    regress2 = Dense(4, activation='sigmoid')(regress1)
    
    detector = Model(inputs=input_layer, outputs=[class2, regress2])
    return detector

### 3.3 Prova de la xarxa neuronal

In [ ]:
detector = build_model()

In [ ]:
X, y = train.as_numpy_iterator().next()

In [ ]:
X.shape

In [ ]:
classes, coords = detector.predict(X)

In [ ]:
classes, coords

### 2.2 Funcions de pèrdua i optimitzadors
Els optimitzadors són algorismes que s’utilitzen per ajustar els pesos d’una xarxa neuronal durant l’entrenament. Els optimitzadors són responsables de minimitzar la funció de pèrdua de la xarxa neuronal.

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.0001) #li introduïm el decay que hem calculat a l'optimitzador

'''
En el context de les xarxes neuronals, un optimitzador és un algorisme
que ajuda a ajustar els paràmetres de la xarxa per aconseguir una millor precisió.
L'optimitzador Adam és un exemple d'això i ajuda a l'optimitzador a convergir més ràpidament
i amb més precisió.
'''

### Creació de 'Localitzation Loss' i 'Classification Loss'

La funció té dos components: la pèrdua de localització i la pèrdua de classificació.
La pèrdua de localització mesura la diferència entre les coordenades dels quadres delimitadors 
predits i les coordenades dels quadres delimitadors reals. La pèrdua de classificació mesura la
diferència entre les probabilitats de classe predites i les probabilitats de classe reals. 
En aquesta funció, només es calcula la pèrdua de localització.

La funció té com a entrada dos tensors: y_true i yhat. 
y_true conté les coordenades dels quadres delimitadors reals i
les probabilitats de classe reals per a cada objecte en la imatge d’entrada.
yhat conté les coordenades dels quadres delimitadors predits i les
probabilitats de classe predites per a cada objecte en la imatge d’entrada.

La funció calcula la pèrdua de localització sumant el quadrat de la diferència
entre les coordenades dels quadres delimitadors reals i les coordenades dels quadres
delimitadors predits. A continuació, calcula la diferència entre l’amplada i l’alçada
dels quadres delimitadors reals i els quadres delimitadors predits i suma els quadrats
d’aquestes diferències. Finalment, retorna la suma de les dues pèrdues.

In [ ]:
def localization_loss(y_true, yhat):#primer valor: coordenades reals, segon valor: coordenades previstes     
    delta_coord = tf.reduce_sum(tf.square(y_true[:,:2] - yhat[:,:2])) #diferència dels dos primers valors de cada fila de la matriu
                  
    h_true = y_true[:,3] - y_true[:,1] #quarta columna d'una matriu - segona columna
    w_true = y_true[:,2] - y_true[:,0] #tercera columna - primera

    h_pred = yhat[:,3] - yhat[:,1] 
    w_pred = yhat[:,2] - yhat[:,0] 
    '''
    delta_size = suma dels quadrats de les diferències entre les dimensions originals 
    i les dimensions reconstruïdes de l'imatge.
    '''
    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true-h_pred))
    return delta_coord + delta_size

In [ ]:
classloss = tf.keras.losses.BinaryCrossentropy() #model que fa una classificació binaria 
regressloss = localization_loss #model que acabem de crear

### Test de les mètriques de 'loss'

In [ ]:
#test, en les 2 cel·les inferiors s'hauria d'obtenir : <tf.Tensor: shape=(), dtype=float32, numpy=n>
regressloss(y[1], coords) #y[0] ==> 0 o 1 (hi ha maduixa o no) , y[1] ==> coords (tot del batch sencer)

In [ ]:
classloss(y[0], classes) #numpy ==> probabilitat

In [ ]:
localization_loss(y[1],coords)

# 3. Entrenament de la intel·ligència artificial

### 3.1 Creació del propi model 

In [ ]:
class Detector(Model): 
    def __init__(self, fruita,  **kwargs): 
        super().__init__(**kwargs)
        self.model = fruita

    def compile(self, opt, classloss, localizationloss, **kwargs):
        super().compile(**kwargs)
        self.closs = classloss
        self.lloss = localizationloss
        self.opt = opt
    '''
    La funció train_step executa cada pas de l'entrenament del model. El model rep un lot de dades
    del entrenament (data que hem escollit per a 'train'),i calcula la pèrdua total (total_loss) que 
    consisteix en la suma de la pèrdua de localització (batch_localizationloss) i la meitat de la
    pèrdua de classificació (batch_classloss). Finalment, calcula la perdua total i actualitza els
    pesos.
    '''
    def train_step(self, batch, **kwargs): 
        
        X, y = batch
        
        with tf.GradientTape() as tape: 
            classes, coords = self.model(X, training=True)
            
            batch_classloss = self.closs(y[0], classes)
            batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
            
            total_loss = batch_localizationloss+0.5*batch_classloss 
            
            grad = tape.gradient(total_loss, self.model.trainable_variables)
            #el gradient representa la direcció i la magnitud en la qual s'ha
            #d'ajustar cada paràmetre del model per reduir la pèrdua (loss) durant l'entrenament
        
        opt.apply_gradients(zip(grad, self.model.trainable_variables))
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
    '''
    Aquesta funció 'test_step' agafa la data de 'train' per avaluar-la i torna a calcular la pèrdua
    total, la pèrdua de classificació i la pèrdua de localització. Això es fa per avaluar el 
    rendiment del model en dades noves.
    '''
    def test_step(self, batch, **kwargs): 
        X, y = batch
        
        classes, coords = self.model(X, training=False)
        
        batch_classloss = self.closs(y[0], classes)
        batch_localizationloss = self.lloss(tf.cast(y[1], tf.float32), coords)
        total_loss = batch_localizationloss+0.5*batch_classloss
        
        return {"total_loss":total_loss, "class_loss":batch_classloss, "regress_loss":batch_localizationloss}
        
    def call(self, X, **kwargs): 
        return self.model(X, **kwargs)

In [ ]:
model = Detector(detector)

El 'mode.compile' configura el model per utilitzar l'optimitzador especificat per minimitzar la combinació de les funcions de pèrdua de classificació i regressió durant el procés d'entrenament

In [ ]:
model.compile(opt, classloss, regressloss)

### 3.2 Entrenament


In [ ]:
logdir='logs' #Crea un directori on es guardarà la informació del Tensorboard 

TensorBoard és una eina de visualització interactiva que s'utilitza en l'entrenament de models de xarxes neuronals per poder entendre millor el comportament del model durant l'entrenament i ajustar els paràmetres de manera més efectiva. 


In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir) #crea un callback per registrar la informació del model 

In [ ]:
hist = model.fit(train, epochs=3, validation_data=val, callbacks=[tensorboard_callback])

Aquesta última línia es passa tota la informació de train (osigui crida a tota la funció d'entrenament) per 10 iteracions (epochs) i registra les dades d'entrenament i validació durant l'entrenament a partir del callback de TensorBoard, llavors "hist" conté tota la info sobre les losses (pèrdues) i l'exactitud del model durant l'entrenament

### 3.3 Anàlisi del rendiment de l'entrenament

In [ ]:
hist.history #per veure totes les pèrdues (losses)

El següent fragment mostra totes les losses de manera gràfica, en teoria tant les pèrdues i les pèrdues validades com les de classificació de localització i les totals haurien de ser valors molt semblants per a ser més exactes, en cas que no ho son deu haver-hi algun problema amb alguna anotation i no l'haurà processat bé (tampoc és un problema greu)

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(20,5))

ax[0].plot(hist.history['total_loss'], color='teal', label='loss')
ax[0].plot(hist.history['val_total_loss'], color='orange', label='val loss')
ax[0].title.set_text('Loss')
ax[0].legend()

ax[1].plot(hist.history['class_loss'], color='teal', label='class loss')
ax[1].plot(hist.history['val_class_loss'], color='orange', label='val class loss')
ax[1].title.set_text('Classification Loss')
ax[1].legend()

ax[2].plot(hist.history['regress_loss'], color='teal', label='regress loss')
ax[2].plot(hist.history['val_regress_loss'], color='orange', label='val regress loss')
ax[2].title.set_text('Regression Loss')
ax[2].legend()

plt.show()

### Provem i desem el model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
Detector.save("detector_fruites.h5") #el desem

### El provem amb la webcam

In [ ]:
Detector = load_model('detector_fruites.h5')

In [ ]:
cap = cv2.VideoCapture(1)
while cap.isOpened():
    _ , frame = cap.read()
    frame = frame[50:500, 50:500,:]
    
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resized = tf.image.resize(rgb, (120,120))
    
    yhat = Detector.predict(np.expand_dims(resized/255,0))
    sample_coords = yhat[1][0]
    
    if yhat[0] > 0.5: 
        # Controls the main rectangle
        cv2.rectangle(frame, 
                      tuple(np.multiply(sample_coords[:2], [450,450]).astype(int)),
                      tuple(np.multiply(sample_coords[2:], [450,450]).astype(int)), 
                            (255,0,0), 2)
        # Controls the label rectangle
        cv2.rectangle(frame, 
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                                    [0,-30])),
                      tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                    [80,0])), 
                            (255,0,0), -1)
        
        if yhat[0] > 0.8 and yhat[0] < 1.2:
        # Controls the text rendered
            cv2.putText(frame, 'poma', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                                [0,-5])),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        elif yhat[0] > 1.8 and yhat[0] < 2.2:
            cv2.putText(frame, 'pera', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                                [0,-5])),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        elif yhat[0] > 2.8 and yhat[0] < 3.2:
                        cv2.putText(frame, 'mandarina', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                                [0,-5])),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        else:           
                cv2.putText(frame, 'no ho tinc clar', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                                                [0,-5])),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    cv2.imshow('EyeTrack', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()